In [2]:
import os
import neo4j
import pandas as pd

path = os.path.abspath(neo4j.__file__)
print(path)

ROOT = os.getcwd().replace('ml/gnn', '')
print(ROOT)

/Users/devpatelio/miniconda3/envs/connexsci/lib/python3.7/site-packages/neo4j/__init__.py
/Users/devpatelio/Downloads/connexsci/


In [78]:
#set up neo4j driver to instance
from neo4j import GraphDatabase
import time

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        print('response sent successfully')
        return response

conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="neo4j",              
                       pwd="iter-v0001")

In [80]:
# Uncomment line after following instructions in pinned slack channel --> 
# Install neo4j desktop obviously
# Create a new project, and then make a local DBMS (set password to iter-v0001 or whatever version number u have running)
# Click on the DBMS and a right hand popup should come, and under plugins install APOC
# Cick the 3 lines beside the DBMS on the initial page, and open terminal and cd into 'conf'
# then type echo 'apoc.import.file.enabled=true\napoc.import.file.use_neo4j_config=true\napoc.export.file.enabled=true' 
# Once installed, start and then open the instance in Neo4J browser (default option)
# In the browser notebook window, under the server status thing, copy the bolt URL
# On the main.ipynb file under gnn (this file), you want to go ahead and run all the cells BUT under the driver class, replace the password and url
# After running the file, the browser should be updated and you will have a local copy of the database
# Run this cell after, and the csv should be found under the 'import' folder


# conn.query(""" CALL apoc.export.csv.all('network.csv', {}) """) 

In [194]:
import numpy as np

DATA = 'ml/data/neo4j-knowledge-graph.csv'
df = pd.read_csv(ROOT+DATA)


paper_db = df[df['_labels'] == ':Paper']
paper_db[paper_db['_id'] == 32503]

df.head()

/Users/devpatelio/miniconda3/envs/connexsci/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (1,2,3,4,5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,_id,_labels,category,id,mesh_id,title,_start,_end,_type
0,0.0,:Category,Gene family,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,:Category,Product (business),NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,:Category,Mental process,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,:Category,Decoy receptors,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,:Category,Modularity (networks),NaN,NaN,NaN,NaN,NaN,NaN


In [186]:
from itertools import chain

class CentralNode():

    def __init__ (self, property, raw, db):
        assert property in ['_id', 'id', 'mesh_id', 'title'], "the property is not specific enough, please provide a unique identifier from ['_id', 'id', 'mesh_id', 'title']"
        get_node = lambda j: db.iloc[j].to_dict()
        node = get_node(db.index[db[property] == raw].tolist()[0])
        # get_rel_ids = lambda pos: [db.iloc[idx]['_end'] for idx, x in enumerate(db[pos]) if node['_id'] == x]
        # get_rel_types = lambda pos: [idx for idx, x in enumerate(db[pos]) if node['_id'] == x]

        # self.rel_ids = [int(x) for x in list(chain(*[get_rel_ids('_start'), get_rel_ids('_end')]))]
        # self.rel_types = [int(x) for x in list(chain(*[get_rel_types('_start'), get_rel_types('_end')]))]

        def directed_edge_node(dir):
            if dir == '_start':
                opp = '_end'
            elif dir == '_end':
                opp = '_start'

            get_rel_inx = lambda pos: [idx for idx, x in enumerate(db[pos]) if node['_id'] == x] 
            rel_indices = [int(x) for x in list(chain(*[get_rel_inx(dir)]))]

            rel_ids = [int(x) for x in [db.iloc[idx][opp] for idx in rel_indices]]
            rel_types = [x for x in [db.iloc[idx]['_type'] for idx in rel_indices]]
            all_nodes = [get_node(rel_id) for rel_id in rel_ids]
            all_edges = [get_node(rel_ix) for rel_ix in rel_indices]

            return [rel_indices, rel_ids, rel_types, all_nodes, all_edges]
        
        self.to_data = directed_edge_node('_start')
        self.from_data = directed_edge_node('_end')

        self.all_ids = {'to': self.to_data[0], 'from': self.from_data[0]}
        self.all_vertices = {'to': self.to_data[3], 'from': self.from_data[3]}
        self.all_vertices['to'].insert(0, node)
        self.all_vertices['from'].insert(0, node)
        self.all_vertices['to'][0]['rel_pos'] = 'central_node'
        self.all_vertices['from'][0]['rel_pos'] = 'central_node'

        self.all_indices = {'to': self.to_data[1], 'from': self.from_data[1]}
        self.all_connections = {'to': self.to_data[4], 'from': self.from_data[4]}
    
    # def get_node_graph(self):
    #     return self.all_nodes, self.all_edges
    
    def get_connected_node(self, index):
        no_central = self.all_rels
        no_central = no_central.pop('')

In [200]:
# df[df['_id'] == 7388]

test = CentralNode('_id', 32503, df)
print(test.all_connections)
print(test.all_vertices)


{'to': [{'_id': nan, '_labels': nan, 'category': nan, 'id': nan, 'mesh_id': nan, 'title': nan, '_start': 32503.0, '_end': 22276.0, '_type': 'IN_CATEGORY'}, {'_id': nan, '_labels': nan, 'category': nan, 'id': nan, 'mesh_id': nan, 'title': nan, '_start': 32503.0, '_end': 23661.0, '_type': 'IN_CATEGORY'}, {'_id': nan, '_labels': nan, 'category': nan, 'id': nan, 'mesh_id': nan, 'title': nan, '_start': 32503.0, '_end': 13428.0, '_type': 'IN_CATEGORY'}, {'_id': nan, '_labels': nan, 'category': nan, 'id': nan, 'mesh_id': nan, 'title': nan, '_start': 32503.0, '_end': 19178.0, '_type': 'IN_CATEGORY'}, {'_id': nan, '_labels': nan, 'category': nan, 'id': nan, 'mesh_id': nan, 'title': nan, '_start': 32503.0, '_end': 22697.0, '_type': 'IN_CATEGORY'}, {'_id': nan, '_labels': nan, 'category': nan, 'id': nan, 'mesh_id': nan, 'title': nan, '_start': 32503.0, '_end': 19379.0, '_type': 'IN_CATEGORY'}, {'_id': nan, '_labels': nan, 'category': nan, 'id': nan, 'mesh_id': nan, 'title': nan, '_start': 32503.0